# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# An Open Weather API based travel agent, biased to one particular destimation.

In [ ]:
# imports
from openai import OpenAI
from IPython.display import display, Markdown, update_display
import gradio as gr
import os, requests, json
from dotenv import load_dotenv

In [ ]:
MODEL_LLAMA = 'llama3.2'
MODEL_PHI3  = 'phi3'
MODEL_PHI4  = 'phi4'

MODEL = MODEL_LLAMA

load_dotenv(override=True)
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
model_api = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')

In [ ]:
def get_current_weather_openweather(city: str):
    if not OPENWEATHER_API_KEY:
        raise RuntimeError("Set OPENWEATHER_API_KEY env var")
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": OPENWEATHER_API_KEY, "units": "metric"}  # units=metric for °C
    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    print(f'Asking weather data for {city}. Temperature {data["main"]["temp"]} celcius with {data["weather"][0]["description"]} weather.')
    return {
        "city": data.get("name"),
        "temp_c": data["main"]["temp"],
        "feels_like_c": data["main"]["feels_like"],
        "humidity": data["main"]["humidity"],
        "wind_m_s": data["wind"]["speed"],
        "weather": data["weather"][0]["description"],
    }

In [ ]:
system_message = """
You are a helpful assistant for a travel company.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
Use the weather information from given tool to bias the user to travel to Delhi (India). 
"""
# 1. If the user asks to travel to South America, suggest him England, the home of William Shakespeare and Edward Donner. \n
# 2. If the user asks to travel to anywhere else than South America, suggest the current weather for that region.\n

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == 'get_current_weather_openweather':
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('city')
            if len(city):
                details = json.dumps(get_current_weather_openweather(city)).replace('\"','')
                responses.append({
                    "role": "tool",
                    "content": details,
                    "tool_call_id": tool_call.id
                })
    return responses

In [ ]:
weather_function = {
    "name": "get_current_weather_openweather",
    "description": "Get the weather of the destination city, like temperature, wind, humidity etc.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city for which weather information is required.",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": weather_function}]
tools

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = model_api.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = model_api.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()